# Dataset QSAR Biodegradation

In [6]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('/Users/anderson/Desktop/proyectos GIT/MLOps/MLFlow-Tracking/qsar-biodeg.csv')

df.head(10)
# df.info()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V33,V34,V35,V36,V37,V38,V39,V40,V41,Class
0,3.919,2.6909,0,0,0,0,0,31.4,2,0,...,0,0,0,2.949,1.591,0,7.253,0,0,2
1,4.170,2.1144,0,0,0,0,0,30.8,1,1,...,0,0,0,3.315,1.967,0,7.257,0,0,2
2,3.932,3.2512,0,0,0,0,0,26.7,2,4,...,0,0,1,3.076,2.417,0,7.601,0,0,2
3,3.000,2.7098,0,0,0,0,0,20.0,0,2,...,0,0,1,3.046,5.000,0,6.690,0,0,2
4,4.236,3.3944,0,0,0,0,0,29.4,2,4,...,0,0,0,3.351,2.405,0,8.003,0,0,2
5,4.236,3.4286,0,0,0,0,0,28.6,2,4,...,0,0,0,3.351,2.556,0,7.904,0,0,2
6,5.000,5.0476,1,0,0,0,0,11.1,0,3,...,0,0,1,4.712,4.583,0,9.303,0,0,2
7,4.525,3.8301,0,0,0,0,0,31.6,3,2,...,0,0,0,3.379,2.143,0,7.950,0,0,2
8,4.596,3.0777,0,0,0,0,2,44.4,2,0,...,0,0,0,3.626,1.917,0,7.939,0,0,2
9,5.040,3.6112,0,0,1,0,2,41.2,0,4,...,1,2,1,3.888,3.500,1,8.706,0,0,2


## Convertir la columna class en binary

In [7]:
# Asegurarse de que no haya valores nulos en la columna Class
df = df.dropna(subset=['Class'])

# Mostrar los valores únicos antes de la conversión
print("🔹 Valores únicos en 'Class' antes de convertir:")
print(df['Class'].unique())

# Si los valores están como texto, convertirlos a enteros
df['Class'] = df['Class'].astype(int)

# Convertir la columna a binaria (1 y 0)
# Si tu dataset usa 1 y 2 como clases:
df['Class'] = df['Class'].map({1: 1, 2: 0})

# Si existen otros valores, puedes forzar binarización con:
# df['Class'] = df['Class'].apply(lambda x: 1 if x != 0 else 0)

# Verificar si quedaron valores NaN después del mapeo
if df['Class'].isna().any():
    print("\n⚠️ Atención: Se encontraron valores no mapeados. Serán reemplazados por 0.")
    df['Class'] = df['Class'].fillna(0).astype(int)

# Mostrar resultados después de la conversión
df.info(10)


🔹 Valores únicos en 'Class' antes de convertir:
[2 1]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1055 entries, 0 to 1054
Data columns (total 42 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      1055 non-null   float64
 1   V2      1055 non-null   float64
 2   V3      1055 non-null   int64  
 3   V4      1055 non-null   int64  
 4   V5      1055 non-null   int64  
 5   V6      1055 non-null   int64  
 6   V7      1055 non-null   int64  
 7   V8      1055 non-null   float64
 8   V9      1055 non-null   int64  
 9   V10     1055 non-null   int64  
 10  V11     1055 non-null   int64  
 11  V12     1055 non-null   float64
 12  V13     1055 non-null   float64
 13  V14     1055 non-null   float64
 14  V15     1055 non-null   float64
 15  V16     1055 non-null   int64  
 16  V17     1055 non-null   float64
 17  V18     1055 non-null   float64
 18  V19     1055 non-null   int64  
 19  V20     1055 non-null   int64  
 20  V21     1055 non-nul

## División de datos de entrenamiento y datos de prueba

In [8]:
X = df.drop('Class', axis=1)
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)

## Estandarización de Datos

In [9]:
scaler = StandardScaler()

# Ajustamos el scaler SÓLO con los datos de entrenamiento
X_train_scaled = scaler.fit_transform(X_train)

# Aplicamos la transformación a los datos de prueba
X_test_scaled = scaler.transform(X_test)

## Creación del experimento en MLflow

In [10]:
import mlflow
mlflow.set_experiment("QSAR_Biodegradation_Experiment")

<Experiment: artifact_location='file:///Users/anderson/Desktop/proyectos%20GIT/MLOps/MLFlow-Tracking/mlruns/354037731558514546', creation_time=1762297336647, experiment_id='354037731558514546', last_update_time=1762297336647, lifecycle_stage='active', name='QSAR_Biodegradation_Experiment', tags={'mlflow.experimentKind': 'custom_model_development'}>

##  Regresión Logística (Scikit-learn)

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import os

with mlflow.start_run(run_name="LR_Manual_v1"):

    params = {
        "C": 0.5,
        "solver": "liblinear",
        "max_iter": 150
    }

    mlflow.log_params(params)

    # Metadatos
    tags = {
        "model_type": "Scikit-learn Logistic Regression",
        "logging_type": "Manual",
        "purpose": "Baseline Model"
    }
    mlflow.set_tags(tags)


    lr = LogisticRegression(
        C=params["C"],
        solver=params["solver"],
        max_iter=params["max_iter"],
        random_state=42
    )
    lr.fit(X_train_scaled, y_train)

    # Evaluación de métricas
    y_pred = lr.predict(X_test_scaled)

    # Calculo de métricas
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1_score": f1_score(y_test, y_pred)
    }
    
    # Registrar las métricas
    mlflow.log_metrics(metrics)
    
    print(f"  Accuracy: {metrics['accuracy']:.4f}")
    print(f"  F1-Score: {metrics['f1_score']:.4f}")

    # Gráficos y Archivos

    # a) Guardar una nota de texto
    mlflow.log_text("Modelo base de Regresión Logística.", 
                   artifact_file="description.txt")

    # b) Guardar un gráfico (Matriz de Confusión)
    fig, ax = plt.subplots()
    ConfusionMatrixDisplay.from_predictions(y_test, y_pred, ax=ax, cmap='Blues')
    plt.title("Matriz de Confusión (LR)")
    
    # Guardamos la figura temporalmente para loggearla
    plot_path = "lr_confusion_matrix.png"
    plt.savefig(plot_path)
    plt.close(fig) # Cerramos la figura

    # Registramos el archivo como artefacto en una subcarpeta "plots"
    mlflow.log_artifact(plot_path, artifact_path="plots")
    
    # (Opcional) Borramos el archivo local
    os.remove(plot_path)

# print("Run de Regresión Logística completado.")

  Accuracy: 0.8436
  F1-Score: 0.8842


## Red Neuronal (TensorFlow/Keras)

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import mlflow.tensorflow

# Según la documentación de MLflow, se debe activar el autologging ANTES de definir/compilar/entrenar el modelo
mlflow.tensorflow.autolog()

# Autologging registrará todo DENTRO de este run
with mlflow.start_run(run_name="NN_TensorFlow_Autolog_v1"):

    # Parámetros, Autologging los detectará del model.fit() y model.compile()
    epochs = 20
    batch_size = 32
    optimizer = 'adam'

    # (Obtener la dimensionalidad de entrada)
    input_dim = X_train_scaled.shape[1] 
    
    # Crear la Red Neuronal con Keras
    model = Sequential([
        # Primera capa oculta con 32 neuronas
        Dense(32, activation='relu', input_shape=(input_dim,)),
        Dropout(0.2), # Esto es para la regularización
        Dense(16, activation='relu'),
        # 1 neurona con 'sigmoid' para clasificación binaria
        Dense(1, activation='sigmoid') 
    ])

    # (Autologging registrará 'optimizer', 'loss' y 'metrics')
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    # Imprimimos un resumen que también será guardado como artefacto
    model.summary()
    print("\nIniciando entrenamiento de la Red Neuronal")

    # (Autologging registrará métricas por época y los resultados finales)
    history = model.fit(
        X_train_scaled,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        # Usamos los datos de test como validación
        validation_data=(X_test_scaled, y_test),
        verbose=1 # Muestra el progreso
    )

# print("Entrenamiento completado.")
# print("El autologging ha registrado todos los resultados.")

/Users/anderson/Desktop/proyectos GIT/MLOps/MLFlow-Tracking/MLFlowTracking/lib/python3.12/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │         1,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,889 (7.38 KB)

 Trainable params: 1,889 (7.38 KB)

...

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8922 - loss: 0.2911 - val_accuracy: 0.8531 - val_loss: 0.3643
Epoch 16/20
19/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8956 - loss: 0.2638 

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8957 - loss: 0.2657 - val_accuracy: 0.8531 - val_loss: 0.3640
Epoch 17/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8934 - loss: 0.2762 - val_accuracy: 0.8531 - val_loss: 0.3660
Epoch 18/20
18/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8786 - loss: 0.3046 

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8957 - loss: 0.2709 - val_accuracy: 0.8626 - val_loss: 0.3627
Epoch 19/20
20/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8966 - loss: 0.2715 

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8934 - loss: 0.2685 - val_accuracy: 0.8578 - val_loss: 0.3584
Epoch 20/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8945 - loss: 0.2631 - val_accuracy: 0.8483 - val_loss: 0.3689
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step


2025/11/06 12:14:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


## Registrar la Interpretación de Ollama en MLflow 

In [14]:
import mlflow

with mlflow.start_run(run_name="Analisis_Ollama"):
    
    mlflow.set_tag("purpose", "Interpretation")
    
    # Registramos el archivo de texto como artefacto
    mlflow.log_artifact("interpretacion_ollama.txt", artifact_path="analysis")

print("Interpretación guardada como artefacto en MLflow.")

Interpretación guardada como artefacto en MLflow.


## Runs Anidados y Autologging Combinado

In [ ]:
import mlflow
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Esta única línea activará el autologging para scikit-learn Y tensorflow
mlflow.autolog(log_models=False) # (log_models=False para que sea más rápido)

print("Se divide en 2 partes: Parte A (Logistic Regression) y Parte B (Red Neuronal)")

# --- Parte A: Búsqueda Anidada con Scikit-learn ---
print("\nParte A: Búsqueda Anidada (Scikit-learn)")

parameters_to_try = [0.1, 0.5, 1.0, 5.0]

# Iniciamos el Run Padre.
with mlflow.start_run(run_name="LR_Hyperparameter_Search") as parent_run:
    
    mlflow.set_tag("purpose", "Nested Runs Example")
    print(f"Iniciando Run Padre: {parent_run.info.run_id}")

    best_f1 = -1
    best_c = None

    # Iterar y crear "Runs Hijos"
    for c_value in parameters_to_try:
        with mlflow.start_run(run_name=f"LR_C_value_{c_value}", nested=True) as child_run:
            print(f"  Iniciando Run Hijo (C={c_value})")
            
            lr = LogisticRegression(C=c_value, solver='liblinear', random_state=42)
            lr.fit(X_train_scaled, y_train)
            
            y_pred = lr.predict(X_test_scaled)
            f1 = f1_score(y_test, y_pred)
            # (Autologging ya registra esto, pero lo usamos para nuestra lógica)
            
            if f1 > best_f1:
                best_f1 = f1
                best_c = c_value

    # Volvemos al Run Padre
    print(f"Mejor resultado LR: C={best_c} con F1-Score={best_f1:.4f}")
    mlflow.log_param("best_C", best_c)
    mlflow.log_metric("best_f1_score", best_f1)

print("--- Parte A Completada ---")

# --- Parte B: Run de Red Neuronal (TensorFlow) ---
print("\nParte B: Run de Red Neuronal (TensorFlow)")

with mlflow.start_run(run_name="NN_TensorFlow_Autolog_v2"):
    
    mlflow.set_tag("purpose", "Combined Autolog Example")
    print("Iniciando Run de Red Neuronal...")

    # Definir Hiperparámetros
    epochs = 15 # Reducimos épocas para que sea más rápido
    batch_size = 32
    input_dim = X_train_scaled.shape[1] 
    
    # Crear la Red Neuronal
    model = Sequential([
        Dense(32, activation='relu', input_shape=(input_dim,)),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid') 
    ])

    # Compilar
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    # Entrenar
    history = model.fit(
        X_train_scaled,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_test_scaled, y_test),
        verbose=0 # Lo ponemos en 0 para no saturar la consola
    )
    
    print("--- Parte B Completada ---")


2025/11/06 12:58:39 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.
2025/11/06 12:58:58 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/11/06 12:58:58 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Se divide en 2 partes: Parte A (Logistic Regression) y Parte B (Red Neuronal)

Parte A: Búsqueda Anidada (Scikit-learn)
Iniciando Run Padre: 381b4304149f43f2821c9989ba7db0d2
  Iniciando Run Hijo (C=0.1)


2025/11/06 12:58:59 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


  Iniciando Run Hijo (C=0.5)


2025/11/06 12:59:00 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


...

2025/11/06 12:59:01 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


  Iniciando Run Hijo (C=5.0)


/Users/anderson/Desktop/proyectos GIT/MLOps/MLFlow-Tracking/MLFlowTracking/lib/python3.12/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025/11/06 12:59:02 WARNING mlflow.keras.autologging: Failed to log dataset information to MLflow. Reason: 'Series' object has no attribute 'flatten'


Mejor resultado LR: C=5.0 con F1-Score=0.8982
--- Parte A Completada ---

Parte B: Run de Red Neuronal (TensorFlow)
Iniciando Run de Red Neuronal...


--- Parte B Completada ---
